In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import panel as pn
pn.extension()

# Load and clean data
df = pd.read_csv('Fall_2025_schedule_standardized.csv')

# Filter for Active courses with TD or BLD methods and required campus
df = df[
    (df['CRSSTATUS'] == 'Active') &
    (df['Inst_Method'].isin(['TD', 'BLD'])) &
    (df['MTGDAYS'].notna()) &
    (df['Standardized_Time'].notna()) &
    (~df['MTGDAYS'].str.upper().eq('TBA')) &
    (~df['Standardized_Time'].str.upper().eq('TBA')) &
    (df['CAMPUS'].isin(['BX', 'DF', 'MT']))
]

# Widgets
campus_select = pn.widgets.Select(name='Campus', options=['All'] + ['BX', 'DF', 'MT'], value='All')
partterm_select = pn.widgets.Select(name='Part Term', options=['All'] + sorted(df['PARTTERM'].dropna().unique()), value='All')

school_options = ['All'] + sorted(df['SCHOOL'].dropna().unique())
school_select = pn.widgets.Select(name='School', options=school_options, value='All')

inst_method_check = pn.widgets.CheckBoxGroup(name='Instructional Method', value=['TD', 'BLD'], options=['TD', 'BLD'], inline=True)
crslevel_check = pn.widgets.CheckBoxGroup(name='Course Level', value=['U', 'G'], options=['U', 'G'], inline=True)

# Plotting function
def plot_heatmap(campus, partterm, school, inst_method, crslevel):
    filtered_df = df.copy()

    if campus != 'All':
        filtered_df = filtered_df[filtered_df['CAMPUS'] == campus]
    if partterm != 'All':
        filtered_df = filtered_df[filtered_df['PARTTERM'] == partterm]
    if school != 'All':
        filtered_df = filtered_df[filtered_df['SCHOOL'] == school]
    if inst_method:
        filtered_df = filtered_df[filtered_df['Inst_Method'].isin(inst_method)]
    if crslevel:
        filtered_df = filtered_df[filtered_df['CRSLEVEL'].isin(crslevel)]

    # Expand days
    days_map = {'M': 'Monday', 'T': 'Tuesday', 'W': 'Wednesday',
                'R': 'Thursday', 'F': 'Friday', 'S': 'Saturday', 'U': 'Sunday'}
    expanded_rows = []
    for _, row in filtered_df.iterrows():
        for d in row['MTGDAYS']:
            if d in days_map:
                expanded_rows.append({
                    'Day': days_map[d],
                    'Standardized_Time': row['Standardized_Time'],
                    'MAXENROLL': row['MAXENROLL']
                })

    heatmap_df = pd.DataFrame(expanded_rows)
    if heatmap_df.empty:
        return pn.pane.Markdown("**No data to display.**")

    # Create pivot table with sum of MAXENROLL
    pivot_table = heatmap_df.groupby(['Standardized_Time', 'Day'])['MAXENROLL'].sum().unstack(fill_value=0)

    # Ensure days are in order
    ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot_table = pivot_table.reindex(columns=ordered_days, fill_value=0)

    # Sort time
    def time_sort_key(time_range):
        try:
            start_time = time_range.split(' - ')[0]
            return datetime.strptime(start_time.strip(), '%I:%M %p')
        except:
            return datetime.strptime('12:00 AM', '%I:%M %p')

    pivot_table = pivot_table.sort_index(key=lambda x: x.map(time_sort_key))

    # Plot
    fig, ax = plt.subplots(figsize=(14, 10))
    sns.heatmap(pivot_table, cmap="YlGnBu", linewidths=.5, annot=True, fmt=".0f", ax=ax)
    ax.set_title(f"Heatmap | Campus: {campus} | PartTerm: {partterm}")
    ax.set_xlabel('Day of Week')
    ax.set_ylabel('Standardized Time')
    plt.tight_layout()

    return pn.pane.Matplotlib(fig, tight=True)

# Bind and layout
interactive_panel = pn.bind(
    plot_heatmap,
    campus=campus_select,
    partterm=partterm_select,
    school=school_select,
    inst_method=inst_method_check,
    crslevel=crslevel_check
)

dashboard = pn.Column(
    "# 📊 Fall 2025 Schedule Heatmap (TD/BLD Courses Only)",
    pn.Row(campus_select, partterm_select),
    pn.Row(school_select),
    pn.Row(inst_method_check),
    pn.Row(crslevel_check),
    interactive_panel
)

dashboard.servable()
